Just some quick resources for wx data while I am thinking of it....
mesowest is a great one. it has all of the government weather stations: http://mesowest.utah.edu/
wunderground has the mpg wx stations: mpgweather.com
More to come!

Next step for wx data: is there a way to automatically download this while we are recording for NFCs. So all the wx stations of interest from start date x to end date x are archived either in real time or not. The archived files are in whatever format for us to import into analysis and analysis software. Think ready to go with little manipulation required by us.
Typical weather variables we are interested in:
temp, wind speed and direction, ceiling height (only available at major airports, I believe), cloud cover, barometric pressure, etc. There maybe more I am forgetting....
As far as radar, I want the Nexrad wx radar from Montana doppler radar sites. I have to track down the websites for this yet. But here is an example of someone who makes maps from available data. http://www.pauljhurtado.com/US_Composite_Radar/
Ultimately, what I want is a csv or txt with the raw radar values from these sorts of images. I want the pixel values at our monitoring sites and then buffered at 300 and 600m from point center and and get focal statistics for each. Does that make sense?

In [258]:
#!/usr/bin/env python2.7
import os, requests, datetime
import pandas as pd
from bs4 import BeautifulSoup

In [250]:
def main():
  for station in station_list:
    # Generate URL
    url = station_url(station)
    
    # Format BeautifulSoup
    soup = read_url(url)
    
    # Parse soup
    weather_values = extract_wx_values(soup, station)
    
    # Combine col_names, wx_values into DataFrame
    df = pd.DataFrame(data=[weather_values], columns=col_names)
    
    # Write to .csv
    write_result(df)
    
if __name__== "__main__":
  main()

In [251]:
station_list = ['KMTFLORE2', 'KMTFLORE7', 'KMTSTEVE13', 
                'KMTSTEVE12', 'KMTFLORE17', 'KMTFLORE18', 'KMTFLORE11', 
                'KMTFLORE12', 'KMTFLORE14', 'KMTVICTO9', 'KMTCORVA12', 
                'KMTCORVA11', 'KMTCORVA15', 'KMTCORVA10', 'KMTLOLO7', 
                'KMTLOLO4', 'KMTLOLO3', 'KMTLOLO2', 'KMTHAMIL13', 
                'KMTHAMIL7', 'KMTSTEVE8', 'KMTCORVA9']

airport_list = ['KMSO']

In [252]:
# column names
col_names = ['date_time', 'station', 'temp', 'feels_like', 'wind_speed', 
           'wind_direction', 'gusts', 'dew_point', 'humidity', 
           'precip_rate', 'accum_precip', 'pressure', 'solar']

In [253]:
# define url
def station_url(station):
    s_url = 'https://www.wunderground.com/personal-weather-station/dashboard?ID=' + station
    return(s_url)

In [254]:
def read_url(url):
    # Make the request and check object type
    r = requests.get(url)

    # Extract HTML from Response object and print
    html = r.text
    
    # Create a BeautifulSoup object from the HTML
    soup = BeautifulSoup(html, "html5lib")
    return(soup)

In [255]:
def extract_wx_values(soup, station):
    wx_values = []
    
    # parse soup for wx_values
    wx_element = soup.findAll('span', class_='wx-value')
    for value in wx_element:
        v = value.contents[0].strip()
        wx_values.append(v)
    
    # add date_time and station
    now = datetime.datetime.now()
    date_time = now.strftime("%Y%m%d %H%M")
    wx_values = [date_time] + [station] + wx_values
    
    # fix for missing solar wx_value
    while len(wx_values) < 13:
        wx_values.append('None')

    return(wx_values)

In [257]:
def write_result(df):
    dirname = './'
    filename = now.strftime("%Y%m%d_%H%M")+ '.csv'

    if os.path.exists(dirname+filename):
        with open(dirname + filename, 'a') as f:
            df.to_csv(f, header=False, index=False)
    else:        
        df.to_csv(dirname + filename, index=False)
        
        #print(dirname+filename)
        #print(df[['station', 'temp', 'date_time', 'solar']])